# Part 1: A Small Example

Investigating and testing the propagating flux ratio method

In [2]:
from functional_testing_tutorial import FireEquations, FireParams, Fuel
FireEquations.optimum_packing_ratio()

ModuleNotFoundError: No module named 'functional_testing_tutorial'